# Reading Files

In [83]:
from os import listdir
import xml.etree.ElementTree as ET

class BoardGame:
    def __init__(self, root):
        self.id = root.attrib["id"].strip()
        nameTags = root.findall("name")
        for n in nameTags:
            if n.attrib["type"] == "primary":
                self.name = n.attrib["value"]
        self.description = ""
        if root.find("description").text != None:
            self.description = root.find("description").text.strip()
        self.year = 0
        if len(root.find("yearpublished").attrib["value"].strip()) != 0:
            self.year = int(root.find("yearpublished").attrib["value"])
        minPlayers = 0
        if len(root.find("minplayers").attrib["value"].strip()) != 0:
            minPlayers = int(root.find("minplayers").attrib["value"])
        maxPlayers = 0
        if len(root.find("maxplayers").attrib["value"].strip()) != 0:
            maxPlayers = int(root.find("maxplayers").attrib["value"])
        self.numPlayer = (minPlayers, maxPlayers)
        self.playTime = 0
        if len(root.find("playingtime").attrib["value"]) != 0:
            self.playTime = int(root.find("playingtime").attrib["value"])
        self.minAge = 0
        if len(root.find("minage").attrib["value"]) != 0:
            self.minAge = int(root.find("minage").attrib["value"])
        self.mechanics = []
        self.family = []
        self.category = []
        self.implementations = []
        xmlLinkLists = root.findall("link")
        for l in xmlLinkLists:
            if l.attrib["type"] == "boardgamemechanic":
                self.mechanics.append((l.attrib["id"].strip(), l.attrib["value"].strip()))
            if l.attrib["type"] == "boardgamecategory":
                self.category.append((l.attrib["id"].strip(), l.attrib["value"].strip()))
            if l.attrib["type"] == "boardgamefamily":
                self.family.append((l.attrib["id"].strip(), l.attrib["value"].strip()))
            if l.attrib["type"] == "boardgameimplementation":
                self.implementations.append(l.attrib["id"].strip())
        self.usersRating = int(root.find("statistics")[0].find("usersrated").attrib["value"])
        self.usersOwned = int(root.find("statistics")[0].find("owned").attrib["value"])
        self.avgRating = float(root.find("statistics")[0].find("average").attrib["value"])
        self.medRating = float(root.find("statistics")[0].find("median").attrib["value"])
        self.stdRating = float(root.find("statistics")[0].find("stddev").attrib["value"])
        self.rankList = []
        xmlRankLists = root.find("statistics")[0].find("ranks")
        for r in xmlRankLists:
            self.rankList.append(r.attrib["friendlyname"].strip())
    def __str__(self):
        return "[" + self.name + "] (" + str(self.year) + ") players:" + str(self.numPlayer) + \
            " time: " + str(self.playTime) + " rating: " + str(self.avgRating)
xmlFiles = [f for f in listdir("GamesXML/") if ".xml" in f]

boardgameDict = {}
mechanicsDict = {}
familyDict = {}
categoryDict = {}
for f in xmlFiles:
    tree = ET.parse("GamesXML/" + f)
    root = tree.getroot()
    for item in root:
        boardgame = BoardGame(item)
        boardgameDict[boardgame.id] = boardgame
        for m in boardgame.mechanics:
            mechanics[m[0]] = m[1]
        for f in boardgame.family:
            family[f[0]] = f[1]
        for c in boardgame.category:
            category[c[0]] = c[1]

In [84]:
def filterUniqueGames(currentList):
    newList = []
    for g in currentList:
        add = True
        for id in g.implementations:
            if id in newDict:
                add = False
                break
        if add:
            newList.append(g)
    return newList

def filterGamesWithMinUsers(currentList, users):
    newList = []
    for g in currentList:
        if g.usersRating > users or g.usersOwned > users:
            newList.append(g)
    return newList

def filterGamesWithYear(currentList, minYear=-1, maxYear=-1):
    newList = []
    for g in currentList:
        if (minYear <= 0 or g.year >= minYear) and (maxYear <= 0 or g.year <= maxYear):
            newList.append(g)
    return newList

def filterGamesWithName(currentList, name):
    newList = []
    for g in currentList:
        if name.strip().lower() in g.name.strip().lower():
            newList.append(g)
    return newList

In [85]:
currentList = boardgameDict.values()
currentList = filterGamesWithYear(currentList, 2019)
currentList = filterGamesWithName(currentList, "dog")
for g in currentList:
    print(g)

[Endogenesis] (2019) players:(1, 5) time: 120 rating: 8.06533
[Pavlov's Dogs] (2020) players:(3, 10) time: 45 rating: 0.0
[We Rate Dogs!: The Card Game] (2019) players:(3, 6) time: 60 rating: 0.0
[Dig Dog Dig] (2019) players:(2, 4) time: 10 rating: 0.0
[Dog Rush] (2019) players:(2, 6) time: 0 rating: 5.53333


In [95]:
def getHist(currentList, attribName, isList=True):
    hist = {}
    for g in currentList:
        value = getattr(g, attribName);
        if isList:
            for v in value:
                if v not in hist:
                    hist[v] = 0
                hist[v] += 1
        else:
            if value not in hist:
                hist[value] = 0
            hist[value] += 1
    return hist

def filterHistOccur(hist, minOccur):
    newHist = {}
    for e in hist:
        if hist[e] >= minOccur:
            newHist[e] = hist[e]
    return newHist

In [119]:
yearHist = filterHistOccur(getHist(boardgameDict.values(), "year", False), 100)
del yearHist[0]
currentList = boardgameDict.values()
attribByYear = {}
for y in yearHist:
    gamesByYear = filterGamesWithYear(currentList, y, y)
    attribHist = getHist(gamesByYear, "mechanics")
    attribList = [(m, attribHist[m]) for m in attribHist]
    attribList = sorted(attribList, key=lambda x: x[1])
    attribByYear[y] = attribList[-1]
attribByYear

{1930: (('2035', 'Roll / Spin and Move'), 58),
 1935: (('2035', 'Roll / Spin and Move'), 33),
 1940: (('2035', 'Roll / Spin and Move'), 45),
 1950: (('2035', 'Roll / Spin and Move'), 55),
 1955: (('2035', 'Roll / Spin and Move'), 65),
 1956: (('2035', 'Roll / Spin and Move'), 65),
 1959: (('2035', 'Roll / Spin and Move'), 66),
 1960: (('2035', 'Roll / Spin and Move'), 97),
 1961: (('2035', 'Roll / Spin and Move'), 49),
 1962: (('2035', 'Roll / Spin and Move'), 54),
 1963: (('2035', 'Roll / Spin and Move'), 64),
 1964: (('2035', 'Roll / Spin and Move'), 68),
 1965: (('2035', 'Roll / Spin and Move'), 76),
 1966: (('2035', 'Roll / Spin and Move'), 56),
 1967: (('2035', 'Roll / Spin and Move'), 57),
 1968: (('2035', 'Roll / Spin and Move'), 62),
 1969: (('2035', 'Roll / Spin and Move'), 67),
 1970: (('2035', 'Roll / Spin and Move'), 83),
 1971: (('2035', 'Roll / Spin and Move'), 66),
 1972: (('2035', 'Roll / Spin and Move'), 66),
 1973: (('2035', 'Roll / Spin and Move'), 95),
 1974: (('203

In [121]:
category

{'1001': 'Political',
 '1002': 'Card Game',
 '1008': 'Nautical',
 '1009': 'Abstract Strategy',
 '1010': 'Fantasy',
 '1011': 'Transportation',
 '1013': 'Farming',
 '1015': 'Civilization',
 '1016': 'Science Fiction',
 '1017': 'Dice',
 '1019': 'Wargame',
 '1020': 'Exploration',
 '1021': 'Economic',
 '1022': 'Adventure',
 '1023': 'Bluffing',
 '1024': 'Horror',
 '1025': 'Word Game',
 '1026': 'Negotiation',
 '1027': 'Trivia',
 '1028': 'Puzzle',
 '1029': 'City Building',
 '1030': 'Party Game',
 '1031': 'Racing',
 '1032': 'Action / Dexterity',
 '1033': 'Mafia',
 '1034': 'Trains',
 '1035': 'Medieval',
 '1036': 'Prehistoric',
 '1037': 'Real-time',
 '1038': 'Sports',
 '1039': 'Deduction',
 '1040': 'Murder/Mystery',
 '1041': "Children's Game",
 '1042': 'Expansion for Base-game',
 '1044': 'Collectible Components',
 '1045': 'Memory',
 '1046': 'Fighting',
 '1047': 'Miniatures',
 '1048': 'American Civil War',
 '1049': 'World War II',
 '1050': 'Ancient',
 '1051': 'Napoleonic',
 '1052': 'Arabian',
 '105